In [5]:
import modelx as mx

model = mx.read_model("Basic_Term_Model_v1")
        

ModuleNotFoundError: No module named 'modelx'

In [3]:
import json
from pathlib import Path
import datetime

def load_settings():
    """Load saved settings from a JSON file"""
    settings_file = Path("saved_settings.json")
    if not settings_file.exists():
        return None
        
    with open(settings_file, "r") as f:
        settings = json.load(f)
        if isinstance(settings.get("valuation_date"), str):
            settings["valuation_date"] = datetime.datetime.strptime(
                settings["valuation_date"], "%Y-%m-%d"
            ).date()
        return settings

load_settings()

{'valuation_date': datetime.date(2025, 1, 15),
 'assumption_table_url': 's3://valuation-model/term/run1/Assumptions.xlsx',
 'projection_period': 20,
 'output_s3_url': 's3://valuation-model/term/run1/output'}

In [1]:

import streamlit as st
import datetime
import pandas as pd

from model_utils import load_assumptions, load_model_points, initialize_model, run_model_calculations, save_results_to_s3, process_all_model_points
from settings_utils import load_settings, save_settings, validate_settings
from log import ModelLogger 
from s3_utils import get_excel_filenames_from_s3

def collect_form_inputs(saved_settings):
    """Collect all form inputs and return settings dict"""
    default_date = datetime.date.today()
    if saved_settings and "valuation_date" in saved_settings:
        try:
            if isinstance(saved_settings["valuation_date"], str):
                default_date = datetime.datetime.strptime(
                    saved_settings["valuation_date"], "%Y-%m-%d"
                ).date()
            else:
                default_date = saved_settings["valuation_date"]
        except (ValueError, TypeError):
            pass
            
    settings = {
        "valuation_date": st.date_input(
            "Valuation Date",
            value=default_date,
            help="Select the valuation date for the pricing model"
        ),
        
        "assumption_table_url": st.text_input(
            "Enter S3 URL for assumption table",
            value=saved_settings.get("assumption_table_url", "") if saved_settings else "",
            help="Format: s3://bucket-name/path/to/file.xlsx"
        ),
    }
    
    # Create a container for model point URL and confirmation
    col1, col2 = st.columns([3, 1], gap="small" , vertical_alignment="bottom")
    
    with col1:
        model_point_url = st.text_input(
            "Enter S3 URL for model point files",
            value=saved_settings.get("model_point_files_url", "") if saved_settings else "",
            help="Format: s3://bucket-name/path/",
            key="mp_url_input"
        )
    with col2:
        confirm_button = st.form_submit_button("Confirm URL")
    
    settings["model_point_files_url"] = model_point_url
    
    # Handle URL confirmation and file listing
    if confirm_button and model_point_url:
        try:
            available_products = get_excel_filenames_from_s3(model_point_url)
            if available_products:
                st.session_state['available_products'] = available_products
            else:
                st.session_state['available_products'] = []
        except Exception as e:
            st.error(f"Error accessing S3 path: {str(e)}")
            st.session_state['available_products'] = []
    
    settings["projection_period"] = st.number_input(
        "Projection Period (Years)",
        min_value=1,
        max_value=100,
        value=saved_settings.get("projection_period", 30) if saved_settings else 30,
        help="Enter the number of years to project"
    )
    
    # Product Groups selection
    available_products = st.session_state.get('available_products', [])
    if available_products:
        selected_products = st.multiselect(
            "Product Groups",
            options=available_products,
            default=saved_settings.get("product_groups", []) if saved_settings else [],
            help="Select product groups to process",
            placeholder="Please select at least one product group"
        )
    else:
        st.multiselect("Product Groups" , options=available_products, help="Confirm model points files URL to show the available products")
        selected_products = []
    
    settings["product_groups"] = selected_products
    
    settings["output_s3_url"] = st.text_input(
        "Enter S3 URL for storing results",
        value=saved_settings.get("output_s3_url", "") if saved_settings else "",
        help="Format: s3://bucket-name/path/to/output/folder/"
    )
    
    return settings

saved_settings = load_settings()

settings = collect_form_inputs(saved_settings)
        

model_points_list = load_model_points(settings["model_point_files_url"])
            

2025-01-29 15:20:39.850 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-29 15:20:39.851 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-29 15:20:39.851 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-29 15:20:39.851 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-29 15:20:39.917 
  command:

    streamlit run /Users/tianying/Library/Python/3.10/lib/python/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-01-29 15:20:39.918 Thread 'MainThread': missing ScriptRunContext! This warning ca

In [9]:
from app import  process_model_run, collect_form_inputs, validate_settings
from model_utils import load_assumptions, load_model_points, initialize_model, run_model_calculations, save_results_to_s3, process_all_model_points, get_available_models
from settings_utils import load_settings, save_settings, validate_settings
from log import ModelLogger 
from s3_utils import get_excel_filenames_from_s3

saved_settings = load_settings()

settings = collect_form_inputs(saved_settings)
validate_settings(settings, validate_required=True)
process_model_run(settings)

2025-01-30 12:53:34.650 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-30 12:53:34.650 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-30 12:53:34.651 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-30 12:53:34.651 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-30 12:53:34.672 
  command:

    streamlit run /Users/tianying/Library/Python/3.10/lib/python/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-01-30 12:53:34.673 Thread 'MainThread': missing ScriptRunContext! This warning ca

ValueError: Settings validation failed: At least one product group must be selected

In [13]:
import pandas as pd
df1 = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6]
})
pd.DataFrame(pd.DataFrame(df1))

,A,B
0,1,4
1,2,5
2,3,6
